In [2]:
import vk_api

from vk_api.longpoll import VkLongPoll, VkEventType
import os.path
from time import time,sleep
import numpy as np
import pandas as pd
import json
from random import randint
global recomendation_profile
from photo_functions import main_photo_id, load_embeddings_1_user_2_photos,find_closest_by_2_photos, recommend_1_photo, write_msg,msg_recomendation
from photo_dictionaries import message_all_func, keyboard_all_func, keyboard_to_photo, \
keyboard_what_to_write_next_5_or_back, keyboard_back, keyboard_next_5_or_back, keyboard_what_to_write,keyboard_back_error,\
message_2_1, message_ask_photo, message_what_to_write, message_end

# API-ключ созданный ранее
token = "c900bb4d9d7657a238e8580eeb91aba9f22df0a811b435bcccfb806bd214c166b845fbf7b9f8f787b17df"
# Авторизуемся как сообщество
vk_group = vk_api.VkApi(token=token)

# Работа с сообщениями
longpoll = VkLongPoll(vk_group)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Model filename: 20170512-110547/20170512-110547.pb
Instructions for updating:
Use tf.gfile.GFile.


In [3]:
%run recomendation_profile/recomendation_profile.py

In [4]:
user_id= 2912995
recomendation_score_2_users(user_id,386503204)

Выгрузка пользователя закончена... :      1.64
Выбор девушек... :      1.66
Предсказание :      1.67
Сбор результата... :      1.69
Расчет окончен... :      1.69


0.0016082860377663968

In [6]:
data_for_predict = pd.read_pickle(df_for_predict_f)

In [12]:
likes_total_1user = pd.read_csv(likes_f, header=None, index_col=0)

In [18]:
import time


In [10]:
loaded_model = joblib.load(model_f)
list_features_correct = list(pd.read_csv(features_for_model_f)['0'].values)
model_info = [loaded_model, list_features_correct]

In [32]:
def predict_1_user(id_search,data_for_predict,files_for_1_user,model_info):
    start=time.monotonic()
    loaded_model,list_features_correct=model_info
    unique_men_df=model_df_for_user_wo_w2v(id_search,files_for_1_user)
    print('Выгрузка пользователя закончена... : {:>9.2f}'.format(time.monotonic()-start))
    if type(unique_men_df)==pd.core.frame.DataFrame:
        try:
            row=data_for_predict.shape[0]
            col=unique_men_df.shape[1]   
            X  = pd.DataFrame(np.repeat(unique_men_df.values,row,axis=1).reshape(col,row)).T
            X=X.join(data_for_predict)
            X.columns=list(unique_men_df)+list(data_for_predict.columns)
            print('Выбор девушек... : {:>9.2f}'.format(time.monotonic()-start))
            X.drop('key',axis=1,inplace=True)
            id_w = X.id_w.values
            X=X[list_features_correct]
            print('Предсказание : {:>9.2f}'.format(time.monotonic()-start))
            y_pred = loaded_model.predict_proba(X)[:,1]
            print('Сбор результата... : {:>9.2f}'.format(time.monotonic()-start))
            user_predict = pd.DataFrame({'id':id_w,'score':y_pred})   
            user_predict.id=user_predict.id.astype(int)
            user_predict['client_id'] = id_search 
            #
            mean_score=user_predict.score.mean()
            #
            user_predict = user_predict.sort_values('score',ascending=False).head(100)
            print('Расчет окончен... : {:>9.2f}'.format(time.monotonic()-start))
            return (user_predict,mean_score)
        except:
            print ('ошибка предикт')
            return(np.nan,np.nan)

In [28]:
mean_score

0.005229705201948866

In [19]:
import time

In [65]:
path_score_2_users = path_to_save + '\\score_2_users.pkl'
path_avg_score=path_to_save + '\\avg_score.pkl'
###
if os.path.exists(path_avg_score):
    avg_score = pd.read_pickle(path_avg_score)
else:
    avg_score=pd.DataFrame({'id_user':[], 'avg_score': []})
    avg_score.to_pickle(path_avg_score)
    
if os.path.exists(path_score_2_users):
    user_score_2_users = pd.read_pickle(path_score_2_users)
else:
    user_score_2_users=pd.DataFrame({'id_user': [], 'id_target':[], 'score': []})
    user_score_2_users.to_pickle(path_score_2_users)
        

In [37]:
path_to_save

'C:\\Users\\pvmek\\Documents\\GitHub\\vk\\Data\\recomendations'

In [72]:
def normalise_score(x,mean_score):
    if x<mean_score: return (x/mean_score*0.5)
    else: return (1/(1+np.exp(-x/mean_score+1)))

In [75]:
def recomendation_score_2_users(id_name,id_target):
    global avg_score,user_score_2_users
    try:
        path_score_2_users = path_to_save + '\\score_2_users.pkl'
        path_avg_score=path_to_save + '\\avg_score.pkl'
        path_user = path_to_save + '\\' + str(id_name) + '.pkl'
        if os.path.exists(path_user):
            mean_score=avg_score.loc[avg_score.id_user==id_name,'avg_score'][0]
        else:
            rec_profile,mean_score=predict_1_user(id_name, data_for_predict, likes_total_1user, model_info)
            rec_profile=rec_profile.id[:100]
            user_recomendations = pd.DataFrame({'rec_profile': rec_profile, 'saw': 0})
            #
            avg_score=avg_score.append(pd.DataFrame({'id_user':[id_name], 'avg_score': [mean_score]}))
            avg_score.to_pickle(path_avg_score)
            user_recomendations.to_pickle(path_user)
        woman_df=model_df_for_user_wo_w2v(id_target,likes_total_1user)
        res,_ = predict_1_user(id_name, woman_df, likes_total_1user, model_info)
        score=normalise_score(res.score[0],mean_score)
        
        score_2_users = pd.DataFrame({'id_user': [id_name], 'id_target':[id_target], 'score': [score]})
        
        user_score_2_users=user_score_2_users.append(score_2_users)
        user_score_2_users.to_pickle(path_score_2_users)
        return (score)
    except:
        return (None)

In [76]:
user_id= 2912995
recomendation_score_2_users(user_id,386503204)

Выгрузка пользователя закончена... :      1.59
Выбор девушек... :      1.59
Предсказание :      1.61
Сбор результата... :      1.62
Расчет окончен... :      1.62


0.15376450255428012

,id_user,avg_score
0,2912995.0,0.00523


In [21]:
res,mean_score=predict_1_user(user_id,data_for_predict,likes_total_1user,model_info)

Выгрузка пользователя закончена... :      1.69
Выбор девушек... :      2.23
Предсказание :      3.53
Сбор результата... :      9.86
Расчет окончен... :      9.91


In [ ]:
#TO DO файлы подгружается на этапа инициализации чат-бота. Убрать говно где я считываю файлы при каждом обращении
# получить коэффициент средний по юзеру
# отнормировать его

In [4]:
predict_1_user(user_id,data_for_predict)

<function functions.predict_1_user(id_search, data_for_predict, files_for_1_user, model_info)>

In [43]:
recomendation_profile_predict(user_id)

Выгрузка пользователя закончена... :      1.67
Выбор девушек... :      2.14
Предсказание :      3.47
Сбор результата... :      9.50
Расчет окончен... :      9.56


241005245

In [ ]:
#user_id= 2912995#msg_recomendation(user_id,keyboard_what_to_write_next_5_or_back,recomendation_profile_predict)

In [ ]:
#score=recomendation_score_2_users(user_id,386503204)
#write_msg(user_id,"Вероятность отношений с этой девушкой составляет "+ str(score)+" процентов",keyboard_back)

In [42]:
def recomendation_profile_predict(id_name):
    global avg_score
    path_user = path_to_save + '\\' + str(id_name) + '.pkl'
    if os.path.exists(path_user):
        user_recomendations = pd.read_pickle(path_user)      
    else:
        rec_profile,mean_score = predict_1_user(id_name, data_for_predict, likes_total_1user, model_info)
        rec_profile=rec_profile.id[:100]
        # rec=['https://vk.com/id'+str(x) for x in rec_profile.values]
        user_recomendations = pd.DataFrame({'rec_profile': rec_profile, 'saw': 0})
        #
        avg_score=avg_score.append(pd.DataFrame({'id_user':[id_name], 'avg_score': [mean_score]}))
        avg_score.to_pickle(path_avg_score)
    if len(user_recomendations.loc[user_recomendations.saw == 0, 'rec_profile'].values)>0:
        results = user_recomendations.loc[user_recomendations.saw == 0, 'rec_profile'].values[0]
        user_recomendations.loc[user_recomendations.rec_profile == results, 'saw'] = 1
        user_recomendations.to_pickle(path_user)
    else:
        user_recomendations.loc[:, 'saw'] = 0
        results = user_recomendations.loc[user_recomendations.saw == 0, 'rec_profile'].values[0]
        user_recomendations.to_pickle(path_user)
    if results is not None:
        return (results)


In [ ]:
dont_stop = True

while dont_stop:
    try:
        # Основной цикл
        for event in longpoll.listen():

            # Если пришло новое сообщение
            if event.type == VkEventType.MESSAGE_NEW:

                # Если это входящее сообщение
                if event.to_me:

                    ev_user_id = event.user_id

                    dialog_history_path = 'Data/Bot_Dialog_History/' + str(ev_user_id) + '.pkl'

                    if os.path.exists(dialog_history_path):
                        dialog_history = pd.read_pickle(dialog_history_path)
                    else:
                        # Если нет файла истории коммуникаций, заводим его
                        dialog_history = pd.DataFrame({'id':[ev_user_id],'time':[int(time())],
                                                       'state':['0'],'message_from_u':[''],
                                                       'meassage_to_u':[''],'target_id':[np.nan]})
                        dialog_history.to_pickle(dialog_history_path)

                    user_id = ev_user_id

                    dialog_history_path = 'Data/Bot_Dialog_History/' + str(user_id) + '.pkl'
                    dialog_history = pd.read_pickle(dialog_history_path)

                    state = dialog_history.state.values[-1] 
                    user_text = event.text # Сообщение от пользователя

                    print(user_text)

                    if state=='0':

                        write_msg(user_id,message_all_func,keyboard_all_func)

                        to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                       'state':['1'],'message_from_u':[user_text],
                                                       'meassage_to_u':['Базовое меню.'],'target_id':[np.nan]})  

                        dialog_history = pd.concat([dialog_history,to_history])
                        dialog_history.to_pickle(dialog_history_path)

                    elif state=='1':

                        if user_text=='Для отношений':
                            target_id=msg_recomendation (user_id,keyboard_what_to_write_next_5_or_back,recomendation_profile_predict)

                            #write_msg(user_id,recomendation_profile(user_id),keyboard_what_to_write_next_5_or_back)
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['2_1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Рекомендация Девушки'],'target_id':[np.nan]})

                        elif user_text=='По фото':

                            write_msg(user_id,message_ask_photo,keyboard_back)

                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['3_1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Просьба отправить фото.'],'target_id':[np.nan]}) 


                        elif user_text=='Выход':

                            write_msg(user_id,message_end)

                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['0'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Прощание.'],'target_id':[np.nan]}) 

                        else:

                            write_msg(user_id,'Пожалуйста, нажми на одну из двух кнопок ниже.',keyboard_all_func)

                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['0'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Просьба нажать на кнопки ниже.'],'target_id':[np.nan]}) 

                        dialog_history = pd.concat([dialog_history,to_history])
                        dialog_history.to_pickle(dialog_history_path)

                    elif state=='2_1':

                        if user_text=="Не нравится":
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                       'state':['2_1'],'message_from_u':[user_text],
                                                       'meassage_to_u':['Не_нравится'],'target_id':[target_id]})

                            target_id=msg_recomendation (user_id,keyboard_what_to_write_next_5_or_back,recomendation_profile_predict)

                            

                        elif user_text=="Нравится": 
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                       'state':['2_1'],'message_from_u':[user_text],
                                                       'meassage_to_u':['Нравится'],'target_id':[target_id]})

                            target_id=msg_recomendation (user_id,keyboard_what_to_write_next_5_or_back,recomendation_profile_predict)

                            

                        elif user_text=="Назад":

                            write_msg(user_id,message_all_func,keyboard_all_func)
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Базовое меню.'],'target_id':[np.nan]})  

                        dialog_history = pd.concat([dialog_history,to_history])
                        dialog_history.to_pickle(dialog_history_path)

                    elif state=='3_1':

                        target_id = user_text.split('/')[-1]

                        cl_photo_path = 'Data/Closest_Photo_Recommedations/' + str(ev_user_id) + '_' + target_id + '.pkl'

                        if os.path.exists(cl_photo_path):
                            # TODO : не рекомендовать для целевой девушки её же профиль
                            # TODO : из закрытого профайла прямым линком скачивать фото
                            to_history = recommend_1_photo(cl_photo_path,user_id,target_id,user_text,keyboard_to_photo)

                        elif user_text=="Назад":

                            write_msg(user_id,message_all_func,keyboard_all_func)
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Базовое меню.'],'target_id':[np.nan]})

                        else:
                            try:
                                target_df = load_embeddings_1_user_2_photos(target_id)
                                if (target_df.len_1[0]==0) and (target_df.len_2[0]!=1):
                                    write_msg(user_id,'Мы не смогли распознать лица на главном фото, пожалуйста, пришли другую ссылку.',keyboard_back)
                                    to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                       'state':['3_1'],'message_from_u':[user_text],
                                       'meassage_to_u':['Не смогли распознать лица'],'target_id':[np.nan]})

                                elif target_df.len_1[0]>1 and target_df.len_2[0]!=1:
                                    write_msg(user_id,'На главном фото видно более одного лица, пожалуйста, пришли другую ссылку.',keyboard_back)
                                    to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                       'state':['3_1'],'message_from_u':[user_text],
                                       'meassage_to_u':['На фото более одного лица'],'target_id':[np.nan]})

                                else:
                                    write_msg(user_id,'Подожди пожалуйста, находим наиболее похожих на эту девушку по фото, \
                                                       это займет 5-10 секунд.')
                                    find_closest_by_2_photos(user_id,target_df).to_pickle(cl_photo_path)
                                    to_history = recommend_1_photo(cl_photo_path,user_id,target_id,user_text,keyboard_to_photo)

                                    #TODO описать ошибку, когда закончится пул девушек

                            except:

                                write_msg(user_id,'К сожалению, мы не смогли распознать ссылку. Пришли, пожалуйста, другую ссылку.',keyboard_back)
                                #TODO сделать кнопку назад

                                to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                       'state':['3_1'],'message_from_u':[user_text],
                                       'meassage_to_u':['Не смогли распознать ссылку'],'target_id':[np.nan]}) 

                        dialog_history = pd.concat([dialog_history,to_history])
                        dialog_history.to_pickle(dialog_history_path)

                    elif state=='3_2': 

                        target_id = dialog_history.target_id.values[-1] # Берем целевую девушку из предыдущего шага

                        if user_text=="1. Что написать?":

                            #TODO к сообщению еще раз приложить ссылку на профиль девушки.

                            vk_group.method('messages.send', {'user_id': user_id, 
                                                'message': message_what_to_write,
                                                'keyboard': json.dumps(keyboard_what_to_write,ensure_ascii=False)})

                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                               'state':['3_2'],'message_from_u':[user_text],
                               'meassage_to_u':['Что можно написать.'],'target_id':[target_id]})

                        elif user_text=="2. Следующую девушку": 

                            cl_photo_path = 'Data/Closest_Photo_Recommedations/' + str(ev_user_id) + '_' + target_id + '.pkl'
                            to_history = recommend_1_photo(cl_photo_path,user_id,target_id,user_text,keyboard_to_photo)

                        elif user_text=="3. Похожие на другую":

                            write_msg(user_id,message_ask_photo)
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['3_1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Просьба отправить фото.'],'target_id':[np.nan]}) 

                        elif user_text=="4. Главное меню":

                            write_msg(user_id,message_all_func,keyboard_all_func)
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Базовое меню.'],'target_id':[np.nan]})   

                        else:
                            #TODO написать что-то в случае набора текста. Пока отправляем следующую девушку.
                            cl_photo_path = 'Data/Closest_Photo_Recommedations/' + str(ev_user_id) + '_' + target_id + '.pkl'
                            to_history = recommend_1_photo(cl_photo_path,user_id,target_id,user_text,keyboard_to_photo)

                        dialog_history = pd.concat([dialog_history,to_history])
                        dialog_history.to_pickle(dialog_history_path)

    except:
        sleep(60)

In [ ]:
dont_stop = True

while dont_stop:
    try:
        # Основной цикл
        for event in longpoll.listen():

            # Если пришло новое сообщение
            if event.type == VkEventType.MESSAGE_NEW:

                # Если это входящее сообщение
                if event.to_me:

                    ev_user_id = event.user_id

                    dialog_history_path = 'Data/Bot_Dialog_History/' + str(ev_user_id) + '.pkl'

                    if os.path.exists(dialog_history_path):
                        dialog_history = pd.read_pickle(dialog_history_path)
                    else:
                        # Если нет файла истории коммуникаций, заводим его
                        dialog_history = pd.DataFrame({'id':[ev_user_id],'time':[int(time())],
                                                       'state':['0'],'message_from_u':[''],
                                                       'meassage_to_u':[''],'target_id':[np.nan]})
                        dialog_history.to_pickle(dialog_history_path)

                    user_id = ev_user_id

                    dialog_history_path = 'Data/Bot_Dialog_History/' + str(user_id) + '.pkl'
                    dialog_history = pd.read_pickle(dialog_history_path)

                    state = dialog_history.state.values[-1] 
                    user_text = event.text # Сообщение от пользователя

                    print(user_text)

                    if state=='0':

                        write_msg(user_id,message_all_func,keyboard_all_func)

                        to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                       'state':['1'],'message_from_u':[user_text],
                                                       'meassage_to_u':['Базовое меню.'],'target_id':[np.nan]})  

                        dialog_history = pd.concat([dialog_history,to_history])
                        dialog_history.to_pickle(dialog_history_path)

                    elif state=='1':

                        if user_text=='Для отношений':
                            target_id=msg_recomendation (user_id,keyboard_what_to_write_next_5_or_back,recomendation_profile_predict)

                            #write_msg(user_id,recomendation_profile(user_id),keyboard_what_to_write_next_5_or_back)
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['2_1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Рекомендация Девушки'],'target_id':[np.nan]})

                        elif user_text=='По фото':

                            write_msg(user_id,message_ask_photo,keyboard_back)

                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['3_1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Просьба отправить фото.'],'target_id':[np.nan]}) 
                        elif user_text=='Вероятность отношений':

                            write_msg(user_id,message_ask_photo,keyboard_back)

                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['4_1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Просьба отправить фото.'],'target_id':[np.nan]}) 


                        elif user_text=='Выход':

                            write_msg(user_id,message_end)

                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['0'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Прощание.'],'target_id':[np.nan]}) 

                        else:

                            write_msg(user_id,'Пожалуйста, нажми на одну из кнопок ниже.',keyboard_all_func)

                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['0'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Просьба нажать на кнопки ниже.'],'target_id':[np.nan]}) 

                        dialog_history = pd.concat([dialog_history,to_history])
                        dialog_history.to_pickle(dialog_history_path)

                    elif state=='2_1':

                        if user_text=="Не нравится":
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                       'state':['2_1'],'message_from_u':[user_text],
                                                       'meassage_to_u':['Не_нравится'],'target_id':[target_id]})

                            target_id=msg_recomendation (user_id,keyboard_what_to_write_next_5_or_back,recomendation_profile_predict)

                            

                        elif user_text=="Нравится": 
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                       'state':['2_1'],'message_from_u':[user_text],
                                                       'meassage_to_u':['Нравится'],'target_id':[target_id]})

                            target_id=msg_recomendation (user_id,keyboard_what_to_write_next_5_or_back,recomendation_profile_predict)

                            

                        elif user_text=="Назад":

                            write_msg(user_id,message_all_func,keyboard_all_func)
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Базовое меню.'],'target_id':[np.nan]})  

                        dialog_history = pd.concat([dialog_history,to_history])
                        dialog_history.to_pickle(dialog_history_path)

                    elif state=='3_1':

                        target_id = user_text.split('/')[-1]

                        cl_photo_path = 'Data/Closest_Photo_Recommedations/' + str(ev_user_id) + '_' + target_id + '.pkl'

                        if os.path.exists(cl_photo_path):
                            # TODO : не рекомендовать для целевой девушки её же профиль
                            # TODO : из закрытого профайла прямым линком скачивать фото
                            to_history = recommend_1_photo(cl_photo_path,user_id,target_id,user_text,keyboard_to_photo)

                        elif user_text=="Назад":

                            write_msg(user_id,message_all_func,keyboard_all_func)
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Базовое меню.'],'target_id':[np.nan]})

                        else:
                            try:
                                target_df = load_embeddings_1_user_2_photos(target_id)
                                if (target_df.len_1[0]==0) and (target_df.len_2[0]!=1):
                                    write_msg(user_id,'Мы не смогли распознать лица на главном фото, пожалуйста, пришли другую ссылку.',keyboard_back)
                                    to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                       'state':['3_1'],'message_from_u':[user_text],
                                       'meassage_to_u':['Не смогли распознать лица'],'target_id':[np.nan]})

                                elif target_df.len_1[0]>1 and target_df.len_2[0]!=1:
                                    write_msg(user_id,'На главном фото видно более одного лица, пожалуйста, пришли другую ссылку.',keyboard_back)
                                    to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                       'state':['3_1'],'message_from_u':[user_text],
                                       'meassage_to_u':['На фото более одного лица'],'target_id':[np.nan]})

                                else:
                                    write_msg(user_id,'Подожди пожалуйста, находим наиболее похожих на эту девушку по фото, \
                                                       это займет 5-10 секунд.')
                                    find_closest_by_2_photos(user_id,target_df).to_pickle(cl_photo_path)
                                    to_history = recommend_1_photo(cl_photo_path,user_id,target_id,user_text,keyboard_to_photo)

                                    #TODO описать ошибку, когда закончится пул девушек

                            except:

                                write_msg(user_id,'К сожалению, мы не смогли распознать ссылку. Пришли, пожалуйста, другую ссылку.',keyboard_back)
                                #TODO сделать кнопку назад

                                to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                       'state':['3_1'],'message_from_u':[user_text],
                                       'meassage_to_u':['Не смогли распознать ссылку'],'target_id':[np.nan]}) 

                        dialog_history = pd.concat([dialog_history,to_history])
                        dialog_history.to_pickle(dialog_history_path)

                    elif state=='3_2': 

                        target_id = dialog_history.target_id.values[-1] # Берем целевую девушку из предыдущего шага

                        if user_text=="1. Что написать?":

                            #TODO к сообщению еще раз приложить ссылку на профиль девушки.

                            vk_group.method('messages.send', {'user_id': user_id, 
                                                'message': message_what_to_write,
                                                'keyboard': json.dumps(keyboard_what_to_write,ensure_ascii=False)})

                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                               'state':['3_2'],'message_from_u':[user_text],
                               'meassage_to_u':['Что можно написать.'],'target_id':[target_id]})

                        elif user_text=="2. Следующую девушку": 

                            cl_photo_path = 'Data/Closest_Photo_Recommedations/' + str(ev_user_id) + '_' + target_id + '.pkl'
                            to_history = recommend_1_photo(cl_photo_path,user_id,target_id,user_text,keyboard_to_photo)

                        elif user_text=="3. Похожие на другую":

                            write_msg(user_id,message_ask_photo)
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['3_1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Просьба отправить фото.'],'target_id':[np.nan]}) 

                        elif user_text=="4. Главное меню":

                            write_msg(user_id,message_all_func,keyboard_all_func)
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Базовое меню.'],'target_id':[np.nan]})   

                        else:
                            #TODO написать что-то в случае набора текста. Пока отправляем следующую девушку.
                            cl_photo_path = 'Data/Closest_Photo_Recommedations/' + str(ev_user_id) + '_' + target_id + '.pkl'
                            to_history = recommend_1_photo(cl_photo_path,user_id,target_id,user_text,keyboard_to_photo)

                        dialog_history = pd.concat([dialog_history,to_history])
                        dialog_history.to_pickle(dialog_history_path)
                    elif state=='4_1':
                        target_id = user_text.split('/')[-1]
                        target_id=vk.users.get(user_ids=target_id)[0]['id']
                        if user_text=="Назад":
                            write_msg(user_id,message_all_func,keyboard_all_func)
                            to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Базовое меню.'],'target_id':[np.nan]})
                        else:
                            try:
                                score=recomendation_score_2_users(user_id,target_id)
                                if score is not None:
                                    write_msg(user_id,"Вероятность отношений с этой девушкой составляет "+ str(score)+" процентов",keyboard_back)
                                    to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                                           'state':['4_1'],'message_from_u':[user_text],
                                                           'meassage_to_u':['Вероятность отношений'],'target_id':[target_id]})
                                else: 
                                    write_msg(user_id,'К сожалению, мы не смогли распознать ссылку либо выгрузить профиль. Пришли, пожалуйста, другую ссылку.',keyboard_back)
                                #TODO сделать кнопку назад

                                to_history = pd.DataFrame({'id':[user_id],'time':[int(time())],
                                       'state':['4_1'],'message_from_u':[user_text],
                                       'meassage_to_u':['Не смогли распознать ссылку или профиль'],'target_id':[np.nan]}) 
                            except:
                                None
                        dialog_history = pd.concat([dialog_history,to_history])
                        dialog_history.to_pickle(dialog_history_path)

    except:
        sleep(60)